In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json
import keras
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 


#from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder



from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConvolution2D, SeparableConv2D
from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation = None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
            self.augmentation.fit(images)

            images = self.augmentation.flow(images, shuffle=False).next()

        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=42)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

In [0]:
!git clone https://github.com/yu4u/cutout-random-erasing.git
!ls -ltr
!pwd

fatal: destination path 'cutout-random-erasing' already exists and is not an empty directory.
total 1844
drwxr-xr-x 2 root root  299008 Dec 13 09:53 resized
-rw-r--r-- 1 root root 1576476 Dec 13 09:58 hvc_annotations.csv
drwxr-xr-x 1 root root    4096 Dec 18 16:52 sample_data
drwx------ 4 root root    4096 Dec 28 09:49 gdrive
drwxr-xr-x 4 root root    4096 Dec 28 09:50 cutout-random-erasing
/content


In [0]:
import os
os.chdir('/content/cutout-random-erasing')
from random_eraser import get_random_eraser
os.chdir('/content')

In [0]:

train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    #augmentation = None
    augmentation=ImageDataGenerator(
        rescale = 1.0 / 255.0,
        preprocessing_function=get_random_eraser(p=0.9, s_l=0.04, s_h=0.2, r_1=0.5, r_2=1/0.3,
                  v_l=0, v_h=255, pixel_level=False)
    )

)


In [0]:
valid_gen = PersonDataGenerator(
    val_df, 
    batch_size=32, 
    shuffle=False,
    augmentation=ImageDataGenerator(
        rescale = 1.0/ 255.0 ) 
)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
import time, math


def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
def Erato_Model(weights=None):
    model = Sequential()

    p = 0.3

    init = 'he_uniform'

    model.add(SeparableConv2D(64, 3, 3, activation= 'relu', border_mode='same',depthwise_initializer =init, input_shape=(224, 224, 3))) 
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(SeparableConv2D(64, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) 
    model.add(BatchNormalization())
    model.add(Dropout(p))

   


    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(p))

    model.add(SeparableConv2D(128, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(SeparableConv2D(128, 3, 3, depthwise_initializer =init, border_mode='same',activation= 'relu')) #13*13*96 and R:14
    model.add(BatchNormalization())
    model.add(Dropout(p))

    


    model.add(MaxPooling2D(pool_size=(2, 2))) # out: 6*6*96 and R:16
    model.add(Dropout(p))

    model.add(SeparableConv2D(256, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(SeparableConv2D(256, 3, 3, depthwise_initializer =init, border_mode='same',activation= 'relu')) #13*13*96 and R:14
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(MaxPooling2D(pool_size=(2, 2))) # out: 6*6*96 and R:16
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same',activation= 'relu')) #13*13*96 and R:14
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(MaxPooling2D(pool_size=(2, 2))) # out: 6*6*96 and R:16
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))


    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same',activation= 'relu')) #13*13*96 and R:14
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3, 3, depthwise_initializer =init, border_mode='same', activation= 'relu')) #out: 15*15*96 and R:10
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(MaxPooling2D(pool_size=(2, 2))) # out: 6*6*96 and R:16
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3,3, depthwise_initializer="he_uniform", activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3,3, depthwise_initializer="he_uniform", activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p))

    model.add(SeparableConv2D(512, 3,3, depthwise_initializer="he_uniform", activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p))


    
    if weights:
        model.load_weights(weights_path)

    return model

In [0]:
backbone = Erato_Model(weights=None)


neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(neck)
    neck = Dense(128)(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="sigmoid", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(inputs=backbone.input, outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion])

In [0]:
#plot_model(model, to_file='model_plot.png')#, show_shapes=True, show_layer_names=True)
#model.load_weights("/content/gdrive/My Drive/SavedModels/checkpoints/PARALLEL_MODEL_val_loss:4.229.hdf5")

In [0]:
model.summary()

In [0]:
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
  return round(0.003 * 1/(1 + 0.319 * epoch), 10)

In [0]:
opt = Adam()
model.compile(
    optimizer=opt,
    loss = "binary_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]

)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
		                                            patience=5,
		                                            verbose=1,
		                                            factor=0.5,
																								
		                                            min_lr=0)

# stop training if no improvements are seen
early_stop = EarlyStopping(monitor="val_loss",
		                           mode="min",
		                           patience=20)

filepath="/content/gdrive/My Drive/SavedModels/checkpoints/parallel_mod_epochs:{epoch:03d}-val_loss:{val_loss:.3f}.hdf5"

# saves model weights to file
checkpoint = ModelCheckpoint(filepath,
		                             monitor='val_loss',
		                             verbose=1,
		                             save_best_only=True,
		                             mode='min',
		                             save_weights_only=False) 

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    callbacks=[learning_rate_reduction, early_stop, checkpoint],#LearningRateScheduler(scheduler, verbose=1),
    verbose=2
)

Epoch 1/50
 - 274s - loss: 4.3376 - gender_output_loss: 0.6953 - image_quality_output_loss: 0.5857 - age_output_loss: 0.4716 - weight_output_loss: 0.4407 - bag_output_loss: 0.5637 - footwear_output_loss: 0.6135 - pose_output_loss: 0.5619 - emotion_output_loss: 0.4053 - gender_output_acc: 0.5478 - image_quality_output_acc: 0.6919 - age_output_acc: 0.7950 - weight_output_acc: 0.8118 - bag_output_acc: 0.6994 - footwear_output_acc: 0.6632 - pose_output_acc: 0.7395 - emotion_output_acc: 0.8532 - val_loss: 4.7531 - val_gender_output_loss: 0.7088 - val_image_quality_output_loss: 0.6023 - val_age_output_loss: 0.5116 - val_weight_output_loss: 0.5023 - val_bag_output_loss: 0.5533 - val_footwear_output_loss: 0.6507 - val_pose_output_loss: 0.5896 - val_emotion_output_loss: 0.6345 - val_gender_output_acc: 0.4477 - val_image_quality_output_acc: 0.7103 - val_age_output_acc: 0.8000 - val_weight_output_acc: 0.8232 - val_bag_output_acc: 0.6706 - val_footwear_output_acc: 0.6215 - val_pose_output_acc: 0.7

# **In the above log the Best RESULT is:**

val_gender_output_acc: 0.7907 - <br/>
val_image_quality_output_acc: 0.5493 - <br/>
val_age_output_acc: 0.7963 - <br/>
val_weight_output_acc: 0.8248 - <br/>
val_bag_output_acc: 0.7216 - <br/>
val_footwear_output_acc: 0.7470 - <br/>
val_pose_output_acc: 0.8161 - <br/>
val_emotion_output_acc: 0.8533


In [0]:
mymodel= keras.models.load_model("/content/gdrive/My Drive/SavedModels/checkpoints/parallel_mod_epochs:023-val_loss:3.535.hdf5")

In [0]:
def evaluate_model(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies

# **Best Result I could achieve form the above CNN Model for this Assignment:**

In [0]:
mymodel_1= keras.models.load_model("/content/gdrive/My Drive/BestModels/epochs:081-val_loss:3.276.hdf5")

In [0]:
p = evaluate_model(mymodel_1)

63/63 [==============================] - 9s 139ms/step


In [0]:
p

{'age_output_acc': 80.0794,
 'bag_output_acc': 77.7943,
 'emotion_output_acc': 85.2803,
 'footwear_output_acc': 78.1085,
 'gender_output_acc': 87.9464,
 'image_quality_output_acc': 73.1482,
 'pose_output_acc': 88.4921,
 'weight_output_acc': 82.1677}